In [1]:
import torch
import pandas as pd
import json
import re
from pathlib import Path
import numpy as np
from PIL import Image
import os
import time
import warnings
import gc
from tqdm import tqdm
BATCH_SIZE = 2 
MAX_NEW_TOKENS = 150
USE_FP16 = True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
BASE_DIR = Path("/kaggle/input/nlp-fashionpedia")
ATTR_CSV = BASE_DIR/"predicted_attributes_final.csv"
IMG_DIR = BASE_DIR/"train_sample/train_sample"
LABELS_JSON = "/kaggle/input/nlp-fashionpedia/label_descriptions.json"
OUTPUT_CSV = "LLaVADescriptions.csv"
CHECKPOINT_FILE = "llava_checkpoint.json"

Using device: cuda


In [4]:
with open(LABELS_JSON) as f:
    label_data = json.load(f)
    attr_id_to_name = {a["id"]: a["name"] for a in label_data["attributes"]}


In [5]:
def build_prompt_llava_v2(attr_vec):
    try:
        if isinstance(attr_vec, str):
            vec = eval(attr_vec, {'np': np, 'int32': np.int32})
        elif isinstance(attr_vec, list):
            vec = attr_vec
        elif isinstance(attr_vec, np.ndarray):
            vec = attr_vec.tolist()
        else:
            vec = list(attr_vec)
        if not isinstance(vec, (list, tuple)):
            raise TypeError(f"Attribute vector is not a list or tuple after processing: {type(vec)} -> {vec}")
    except Exception as e:
        names = ["attribute data error"]
    else:
        names = [attr_id_to_name.get(i) for i, v in enumerate(vec) if v == 1 and attr_id_to_name.get(i)]
        names = [re.sub(r"\s*\(.*?\)", "", n).strip() for n in names if n]
    attr_str = ", ".join(names) if names else "no specific attributes detected"
    return (
      "USER: <image>\n"
      "Describe this clothing item in detail. Do not mention words like lady, woman, man, model, etc. "
      f"It has the following characteristics: {attr_str}. Focus on its visual appearance, including the color, material, any patterns, the style of the garment (e.g., casual, formal), and specific features like the neckline, sleeves, etc., and overall silhouette. "
      "ASSISTANT:"
    )


In [6]:
def clean_up_memory():

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def load_checkpoint():

    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {"processed_ids": [], "descriptions": []}

In [7]:
def save_checkpoint(checkpoint_data):
   
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint_data, f)

def preprocess_image(img_path, target_size=(336, 336)):
   
    try:
        img = Image.open(img_path).convert("RGB")
        img = img.resize(target_size, Image.LANCZOS)
        return img
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")


In [9]:
from transformers import AutoProcessor, AutoModelForVision2Seq

model_id = "llava-hf/llava-1.5-13b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16 if USE_FP16 else torch.float32,
    device_map="auto",
)

model.gradient_checkpointing_enable()

checkpoint = load_checkpoint()
processed_ids = set(checkpoint["processed_ids"])
generated = checkpoint["descriptions"]
df = pd.read_csv(ATTR_CSV)
df = df[~df["image_id"].isin(processed_ids)]

print(f"Found {len(df)} images to process")

num_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE

2025-04-17 13:19:08.474219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744895948.716881      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744895948.784705      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Found 2000 images to process


In [10]:
try:
    for batch_idx in tqdm(range(num_batches), desc="Generating"):
        
        clean_up_memory()
        
        batch_df = df.iloc[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]
        images, prompts, ids = [], [], []
        
        for _, row in batch_df.iterrows():
            img_id = row["image_id"]
            img_path = Path(IMG_DIR) / f"{img_id}.jpg"
            
            img = preprocess_image(img_path)
            if img is None:
                continue
            
            images.append(img)
            prompts.append(build_prompt_llava_v2(row["attributes"]))
            ids.append(img_id)
        
        if not images:
            continue
        
        try:
            with torch.no_grad():
                inputs = processor(
                    images=images,
                    text=prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True
                ).to(device)
                
                
                with torch.cuda.amp.autocast(enabled=USE_FP16):
                    gen_ids = model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        do_sample=False,
                        use_cache=True
                    )
            
            outputs = processor.batch_decode(gen_ids, skip_special_tokens=True)
            

            for img_id, raw_output in zip(ids, outputs):
               
                if "ASSISTANT:" in raw_output:
                    description = raw_output.split("ASSISTANT:", 1)[1].strip()
                else:
                    description = raw_output
                

                sentences = re.split(r'(?<=[.!?])\s+', description)
                if sentences and not re.search(r'[.!?]$', sentences[-1]):
                    
                    if len(sentences) > 1:
                        description = ' '.join(sentences[:-1])
                    else:
                        description = sentences[0] + "."
                
                generated.append({"image_id": img_id, "description": description})
                processed_ids.add(img_id)
            
            del inputs, gen_ids, outputs
            clean_up_memory()
            
            if batch_idx % 5 == 0 or batch_idx == num_batches - 1:
                checkpoint = {
                    "processed_ids": list(processed_ids),
                    "descriptions": generated
                }
                save_checkpoint(checkpoint)
                
                
                temp_df = pd.DataFrame(generated)
                temp_df.to_csv(OUTPUT_CSV, index=False)
                print(f"✓ Checkpoint saved: {len(generated)} descriptions processed")
                
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"OOM error in batch {batch_idx}. Saving checkpoint and exiting.")
                checkpoint = {
                    "processed_ids": list(processed_ids),
                    "descriptions": generated
                }
                save_checkpoint(checkpoint)
            
                temp_df = pd.DataFrame(generated)
                temp_df.to_csv(OUTPUT_CSV, index=False)
                print(f"✓ Saved {len(generated)} descriptions before OOM error")
                break

except KeyboardInterrupt:
    checkpoint = {
        "processed_ids": list(processed_ids),
        "descriptions": generated
    }
    save_checkpoint(checkpoint)

finally:
    out_df = pd.DataFrame(generated)
    out_df.to_csv(OUTPUT_CSV, index=False)
    print(f" Wrote {len(out_df)} descriptions to {OUTPUT_CSV}")

Generating:   0%|          | 0/1000 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_31/563043437.py:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=USE_FP16):
Generating:   0%|          | 1/1000 [00:22<6:07:57, 22.10s/it]

✓ Checkpoint saved: 2 descriptions processed


Generating:   1%|          | 6/1000 [01:55<5:01:44, 18.21s/it]

✓ Checkpoint saved: 12 descriptions processed


Generating:   1%|          | 11/1000 [03:28<5:12:58, 18.99s/it]

...

✓ Checkpoint saved: 1982 descriptions processed


Generating: 100%|█████████▉| 996/1000 [5:15:02<01:16, 19.11s/it]

✓ Checkpoint saved: 1992 descriptions processed


Generating: 100%|██████████| 1000/1000 [5:16:16<00:00, 18.98s/it]

✓ Checkpoint saved: 2000 descriptions processed
✔️  Wrote 2000 descriptions to LLaVADescriptions.csv
